In [1]:
import time
import warnings
import torch
import torch.nn.functional as F
from torch_geometric.nn.models.basic_gnn import GCN, GAT, MLP
from models import GEE

In [2]:
import argparse

def parse_args():
    """
    Generate a parameters parser.
    """
    # parse parameters
    parser = argparse.ArgumentParser()

    # main parameters
    parser.add_argument('--seed', type=int, default=3407, 
                        help='Random seed.')
    
    return parser.parse_args()

# args = parse_args()

In [3]:
args = argparse.Namespace()

args.model = 'gcn'
args.dataset = 'texas'
args.hidden_dim = 32
args.dropout = 0.5
args.layers = 2
args.lr = 0.01
args.weight_decay = 5e-4
args.epochs = 100
args.sampled_nodes = 5
args.heads = 8

In [4]:
import utils

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=Warning)
    graph, adj, features, labels, idx_train, idx_val, idx_test = utils.get_dataset(args.dataset)

Done loading data from cached files.


In [5]:
if(args.model == 'gcn'):
    model = GCN(in_channels=features.shape[1],
                hidden_channels=args.hidden_dim,
                num_layers = args.layers,
                out_channels=labels.max().item() + 1,
                dropout=args.dropout,
                act = 'relu',
                jk = 'cat')
elif(args.model == 'gat'):
    model = GAT(in_channels=features.shape[1],
                hidden_channels=args.hidden_dim,
                num_layers = args.layers,
                out_channels=labels.max().item() + 1,
                dropout=args.dropout,
                heads = args.heads,
                act = 'relu',
                jk = 'cat')
elif(args.model == 'gee'):

    features = torch.cat((features, GEE(labels, adj)), dim=1)

    model = MLP(in_channels=features.shape[1],
                hidden_channels=args.hidden_dim,
                num_layers = args.layers,
                out_channels=labels.max().item() + 1,
                dropout=args.dropout,
                act = 'relu')


optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)


In [6]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(features, adj)
    loss = F.cross_entropy(out[idx_train], labels[idx_train])
    loss.backward()
    optimizer.step()

    return float(loss)

@torch.no_grad()
def test():
    model.eval()
    pred = model(features, adj).argmax(dim=-1)

    accs = []
    for mask in [idx_train, idx_val, idx_test]:
        accs.append( int((pred[mask] == labels[mask]).sum()) / int(mask.sum()) )

    return accs

In [7]:
begin = time.time()

best_val_acc = test_acc = 0
for epoch in range(1, args.epochs + 1):
    loss = train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, '
          f'Val: {val_acc:.4f}')

print("Optimization Finished!")
print("Training time: {:.4f}s".format(time.time() - begin))
print("Best validation accuracy: {:.4f}".format(best_val_acc))
print("Test accuracy: {:.4f}".format(test_acc))

Epoch: 001, Loss: 1.6037, Train: 0.5287, Val: 0.5254
Epoch: 002, Loss: 1.3811, Train: 0.5287, Val: 0.5254
Epoch: 003, Loss: 1.2254, Train: 0.5402, Val: 0.5254
Epoch: 004, Loss: 1.0923, Train: 0.6897, Val: 0.5763
Epoch: 005, Loss: 0.9827, Train: 0.7356, Val: 0.5593
Epoch: 006, Loss: 1.0403, Train: 0.7471, Val: 0.5593
Epoch: 007, Loss: 0.9318, Train: 0.7356, Val: 0.5593
Epoch: 008, Loss: 0.9455, Train: 0.7816, Val: 0.5424
Epoch: 009, Loss: 0.7816, Train: 0.8276, Val: 0.5763
Epoch: 010, Loss: 0.7523, Train: 0.8621, Val: 0.5932
Epoch: 011, Loss: 0.7206, Train: 0.8736, Val: 0.5763
Epoch: 012, Loss: 0.6413, Train: 0.8736, Val: 0.5763
Epoch: 013, Loss: 0.5401, Train: 0.8736, Val: 0.5763
Epoch: 014, Loss: 0.5613, Train: 0.8736, Val: 0.5763
Epoch: 015, Loss: 0.5753, Train: 0.8851, Val: 0.5932
Epoch: 016, Loss: 0.4996, Train: 0.8966, Val: 0.5932
Epoch: 017, Loss: 0.5077, Train: 0.8851, Val: 0.5932
Epoch: 018, Loss: 0.4831, Train: 0.8966, Val: 0.5932
Epoch: 019, Loss: 0.4755, Train: 0.9080, Val: 

In [11]:
!python train.py --dataset=cora --model=gcn --dropout=0.85 --epochs=100 --hidden_dim=512 --layers=3 --lr=0.008 --weight_decay=0.009
!python train.py --dataset=cora --model=gcn --dropout=0.85 --epochs=100 --hidden_dim=512 --layers=3 --lr=0.008 --weight_decay=0.009
!python train.py --dataset=cora --model=gcn --dropout=0.85 --epochs=100 --hidden_dim=512 --layers=3 --lr=0.008 --weight_decay=0.009
!python train.py --dataset=cora --model=gcn --dropout=0.85 --epochs=100 --hidden_dim=512 --layers=3 --lr=0.008 --weight_decay=0.009
!python train.py --dataset=cora --model=gcn --dropout=0.85 --epochs=100 --hidden_dim=512 --layers=3 --lr=0.008 --weight_decay=0.009
!python train.py --dataset=cora --model=gcn --dropout=0.85 --epochs=100 --hidden_dim=512 --layers=3 --lr=0.008 --weight_decay=0.009
!python train.py --dataset=cora --model=gcn --dropout=0.85 --epochs=100 --hidden_dim=512 --layers=3 --lr=0.008 --weight_decay=0.009
!python train.py --dataset=cora --model=gcn --dropout=0.85 --epochs=100 --hidden_dim=512 --layers=3 --lr=0.008 --weight_decay=0.009
!python train.py --dataset=cora --model=gcn --dropout=0.85 --epochs=100 --hidden_dim=512 --layers=3 --lr=0.008 --weight_decay=0.009
!python train.py --dataset=cora --model=gcn --dropout=0.85 --epochs=100 --hidden_dim=512 --layers=3 --lr=0.008 --weight_decay=0.009

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Epoch: 010, Loss: 1.6896, Train: 0.8000, Val: 0.4720
Epoch: 020, Loss: 0.3803, Train: 0.9929, Val: 0.8100
Epoch: 030, Loss: 0.0834, Train: 1.0000, Val: 0.7840
Epoch: 040, Loss: 0.0351, Train: 1.0000, Val: 0.7820
Epoch: 050, Loss: 0.0130, Train: 1.0000, Val: 0.7700
Epoch: 060, Loss: 0.0397, Train: 1.0000, Val: 0.7880
Epoch: 070, Loss: 0.0030, Train: 1.0000, Val: 0.7740
Epoch: 080, Loss: 0.0024, Train: 1.0000, Val: 0.7640
Epoch: 090, Loss: 0.0009, Train: 1.0000, Val: 0.7840
Epoch: 100, Loss: 0.0089, Train: 1.0000, Val: 0.7800
Optimization Finished!
Training time: 69.0498s
Best validation accuracy: 0.8100
Test accuracy: 0.8230
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Epoc

In [12]:
!python train.py --dataset=cora --model=gat --dropout=0.5 --epochs=100 --hidden_dim=512 --layers=3 --lr=0.007 --weight_decay=0.0015 --heads=32
!python train.py --dataset=cora --model=gat --dropout=0.5 --epochs=100 --hidden_dim=512 --layers=3 --lr=0.007 --weight_decay=0.0015 --heads=32
!python train.py --dataset=cora --model=gat --dropout=0.5 --epochs=100 --hidden_dim=512 --layers=3 --lr=0.007 --weight_decay=0.0015 --heads=32
!python train.py --dataset=cora --model=gat --dropout=0.5 --epochs=100 --hidden_dim=512 --layers=3 --lr=0.007 --weight_decay=0.0015 --heads=32
!python train.py --dataset=cora --model=gat --dropout=0.5 --epochs=100 --hidden_dim=512 --layers=3 --lr=0.007 --weight_decay=0.0015 --heads=32
!python train.py --dataset=cora --model=gat --dropout=0.5 --epochs=100 --hidden_dim=512 --layers=3 --lr=0.007 --weight_decay=0.0015 --heads=32
!python train.py --dataset=cora --model=gat --dropout=0.5 --epochs=100 --hidden_dim=512 --layers=3 --lr=0.007 --weight_decay=0.0015 --heads=32
!python train.py --dataset=cora --model=gat --dropout=0.5 --epochs=100 --hidden_dim=512 --layers=3 --lr=0.007 --weight_decay=0.0015 --heads=32
!python train.py --dataset=cora --model=gat --dropout=0.5 --epochs=100 --hidden_dim=512 --layers=3 --lr=0.007 --weight_decay=0.0015 --heads=32
!python train.py --dataset=cora --model=gat --dropout=0.5 --epochs=100 --hidden_dim=512 --layers=3 --lr=0.007 --weight_decay=0.0015 --heads=32

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Epoch: 010, Loss: 0.5933, Train: 0.9429, Val: 0.7660
Epoch: 020, Loss: 0.0171, Train: 1.0000, Val: 0.7840
Epoch: 030, Loss: 0.0191, Train: 1.0000, Val: 0.8060
Epoch: 040, Loss: 0.0005, Train: 1.0000, Val: 0.7780
Epoch: 050, Loss: 0.0376, Train: 1.0000, Val: 0.7340
Epoch: 060, Loss: 0.0011, Train: 1.0000, Val: 0.7480
Epoch: 070, Loss: 0.0001, Train: 1.0000, Val: 0.7500
Epoch: 080, Loss: 0.0003, Train: 1.0000, Val: 0.7340
Epoch: 090, Loss: 0.0004, Train: 1.0000, Val: 0.7520
Epoch: 100, Loss: 0.0007, Train: 1.0000, Val: 0.7340
Optimization Finished!
Training time: 71.9878s
Best validation accuracy: 0.8060
Test accuracy: 0.8120
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Epoc

In [13]:
!python train.py --dataset=cora --model=gee --dropout=0.5 --epochs=100 --hidden_dim=256 --layers=1 --lr=0.005 --weight_decay=0.004
!python train.py --dataset=cora --model=gee --dropout=0.5 --epochs=100 --hidden_dim=256 --layers=1 --lr=0.005 --weight_decay=0.004
!python train.py --dataset=cora --model=gee --dropout=0.5 --epochs=100 --hidden_dim=256 --layers=1 --lr=0.005 --weight_decay=0.004
!python train.py --dataset=cora --model=gee --dropout=0.5 --epochs=100 --hidden_dim=256 --layers=1 --lr=0.005 --weight_decay=0.004
!python train.py --dataset=cora --model=gee --dropout=0.5 --epochs=100 --hidden_dim=256 --layers=1 --lr=0.005 --weight_decay=0.004
!python train.py --dataset=cora --model=gee --dropout=0.5 --epochs=100 --hidden_dim=256 --layers=1 --lr=0.005 --weight_decay=0.004
!python train.py --dataset=cora --model=gee --dropout=0.5 --epochs=100 --hidden_dim=256 --layers=1 --lr=0.005 --weight_decay=0.004
!python train.py --dataset=cora --model=gee --dropout=0.5 --epochs=100 --hidden_dim=256 --layers=1 --lr=0.005 --weight_decay=0.004
!python train.py --dataset=cora --model=gee --dropout=0.5 --epochs=100 --hidden_dim=256 --layers=1 --lr=0.005 --weight_decay=0.004
!python train.py --dataset=cora --model=gee --dropout=0.5 --epochs=100 --hidden_dim=256 --layers=1 --lr=0.005 --weight_decay=0.004

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Epoch: 010, Loss: 1.8949, Train: 1.0000, Val: 0.3940
Epoch: 020, Loss: 1.8392, Train: 1.0000, Val: 0.5220
Epoch: 030, Loss: 1.7845, Train: 1.0000, Val: 0.5220
Epoch: 040, Loss: 1.7307, Train: 1.0000, Val: 0.5300
Epoch: 050, Loss: 1.6779, Train: 1.0000, Val: 0.5240
Epoch: 060, Loss: 1.6263, Train: 1.0000, Val: 0.5240
Epoch: 070, Loss: 1.5757, Train: 1.0000, Val: 0.5180
Epoch: 080, Loss: 1.5263, Train: 1.0000, Val: 0.5180
Epoch: 090, Loss: 1.4780, Train: 1.0000, Val: 0.5160
Epoch: 100, Loss: 1.4309, Train: 1.0000, Val: 0.5160
Optimization Finished!
Training time: 0.8524s
Best validation accuracy: 0.5340
Test accuracy: 0.4770
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Epoch

In [14]:
!python train.py --dataset=pubmed --model=gcn --dropout=0.1 --epochs=100 --hidden_dim=128 --layers=4 --lr=0.001 --weight_decay=0.009
!python train.py --dataset=pubmed --model=gcn --dropout=0.1 --epochs=100 --hidden_dim=128 --layers=4 --lr=0.001 --weight_decay=0.009
!python train.py --dataset=pubmed --model=gcn --dropout=0.1 --epochs=100 --hidden_dim=128 --layers=4 --lr=0.001 --weight_decay=0.009
!python train.py --dataset=pubmed --model=gcn --dropout=0.1 --epochs=100 --hidden_dim=128 --layers=4 --lr=0.001 --weight_decay=0.009
!python train.py --dataset=pubmed --model=gcn --dropout=0.1 --epochs=100 --hidden_dim=128 --layers=4 --lr=0.001 --weight_decay=0.009
!python train.py --dataset=pubmed --model=gcn --dropout=0.1 --epochs=100 --hidden_dim=128 --layers=4 --lr=0.001 --weight_decay=0.009
!python train.py --dataset=pubmed --model=gcn --dropout=0.1 --epochs=100 --hidden_dim=128 --layers=4 --lr=0.001 --weight_decay=0.009
!python train.py --dataset=pubmed --model=gcn --dropout=0.1 --epochs=100 --hidden_dim=128 --layers=4 --lr=0.001 --weight_decay=0.009
!python train.py --dataset=pubmed --model=gcn --dropout=0.1 --epochs=100 --hidden_dim=128 --layers=4 --lr=0.001 --weight_decay=0.009
!python train.py --dataset=pubmed --model=gcn --dropout=0.1 --epochs=100 --hidden_dim=128 --layers=4 --lr=0.001 --weight_decay=0.009

  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Epoch: 010, Loss: 1.0797, Train: 0.5500, Val: 0.3280
Epoch: 020, Loss: 0.9887, Train: 0.8167, Val: 0.6060
Epoch: 030, Loss: 0.7250, Train: 0.9167, Val: 0.7160
Epoch: 040, Loss: 0.3802, Train: 0.9333, Val: 0.7480
Epoch: 050, Loss: 0.1249, Train: 0.9667, Val: 0.7660
Epoch: 060, Loss: 0.0339, Train: 1.0000, Val: 0.7700
Epoch: 070, Loss: 0.0076, Train: 1.0000, Val: 0.7720
Epoch: 080, Loss: 0.0038, Train: 1.0000, Val: 0.7720
Epoch: 090, Loss: 0.0022, Train: 1.0000, Val: 0.7660
Epoch: 100, Loss: 0.0016, Train: 1.0000, Val: 0.7740
Optimization Finished!
Training time: 116.0246s
Best validation accuracy: 0.7820
Test accuracy: 0.7640
  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Epoch

In [22]:
!python train.py --dataset=pubmed --model=gat --dropout=0.3 --epochs=100 --hidden_dim=256 --layers=4 --lr=0.008 --weight_decay=0.004 --heads=16
!python train.py --dataset=pubmed --model=gat --dropout=0.3 --epochs=100 --hidden_dim=256 --layers=4 --lr=0.008 --weight_decay=0.004 --heads=16
!python train.py --dataset=pubmed --model=gat --dropout=0.3 --epochs=100 --hidden_dim=256 --layers=4 --lr=0.008 --weight_decay=0.004 --heads=16
!python train.py --dataset=pubmed --model=gat --dropout=0.3 --epochs=100 --hidden_dim=256 --layers=4 --lr=0.008 --weight_decay=0.004 --heads=16
!python train.py --dataset=pubmed --model=gat --dropout=0.3 --epochs=100 --hidden_dim=256 --layers=4 --lr=0.008 --weight_decay=0.004 --heads=16
!python train.py --dataset=pubmed --model=gat --dropout=0.3 --epochs=100 --hidden_dim=256 --layers=4 --lr=0.008 --weight_decay=0.004 --heads=16
!python train.py --dataset=pubmed --model=gat --dropout=0.3 --epochs=100 --hidden_dim=256 --layers=4 --lr=0.008 --weight_decay=0.004 --heads=16
!python train.py --dataset=pubmed --model=gat --dropout=0.3 --epochs=100 --hidden_dim=256 --layers=4 --lr=0.008 --weight_decay=0.004 --heads=16
!python train.py --dataset=pubmed --model=gat --dropout=0.3 --epochs=100 --hidden_dim=256 --layers=4 --lr=0.008 --weight_decay=0.004 --heads=16
!python train.py --dataset=pubmed --model=gat --dropout=0.3 --epochs=100 --hidden_dim=256 --layers=4 --lr=0.008 --weight_decay=0.004 --heads=16

  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Epoch: 010, Loss: 0.2202, Train: 0.9167, Val: 0.7060
Epoch: 020, Loss: 0.0416, Train: 0.9833, Val: 0.7840
Epoch: 030, Loss: 0.0088, Train: 1.0000, Val: 0.7480
Epoch: 040, Loss: 0.0038, Train: 1.0000, Val: 0.7540
Epoch: 050, Loss: 0.0004, Train: 1.0000, Val: 0.7640
Epoch: 060, Loss: 0.0002, Train: 1.0000, Val: 0.7520
Epoch: 070, Loss: 0.0000, Train: 1.0000, Val: 0.7580
Epoch: 080, Loss: 0.0000, Train: 1.0000, Val: 0.7540
Epoch: 090, Loss: 0.0000, Train: 1.0000, Val: 0.7540
Epoch: 100, Loss: 0.0000, Train: 1.0000, Val: 0.7600
Optimization Finished!
Training time: 333.9422s
Best validation accuracy: 0.7960
Test accuracy: 0.7770
  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Epoch

In [19]:
!python train.py --dataset=pubmed --model=gee --dropout=0.9 --epochs=100 --hidden_dim=256 --layers=3 --lr=0.005 --weight_decay=0.009
!python train.py --dataset=pubmed --model=gee --dropout=0.9 --epochs=100 --hidden_dim=256 --layers=3 --lr=0.005 --weight_decay=0.009
!python train.py --dataset=pubmed --model=gee --dropout=0.9 --epochs=100 --hidden_dim=256 --layers=3 --lr=0.005 --weight_decay=0.009
!python train.py --dataset=pubmed --model=gee --dropout=0.9 --epochs=100 --hidden_dim=256 --layers=3 --lr=0.005 --weight_decay=0.009
!python train.py --dataset=pubmed --model=gee --dropout=0.9 --epochs=100 --hidden_dim=256 --layers=3 --lr=0.005 --weight_decay=0.009
!python train.py --dataset=pubmed --model=gee --dropout=0.9 --epochs=100 --hidden_dim=256 --layers=3 --lr=0.005 --weight_decay=0.009
!python train.py --dataset=pubmed --model=gee --dropout=0.9 --epochs=100 --hidden_dim=256 --layers=3 --lr=0.005 --weight_decay=0.009
!python train.py --dataset=pubmed --model=gee --dropout=0.9 --epochs=100 --hidden_dim=256 --layers=3 --lr=0.005 --weight_decay=0.009
!python train.py --dataset=pubmed --model=gee --dropout=0.9 --epochs=100 --hidden_dim=256 --layers=3 --lr=0.005 --weight_decay=0.009
!python train.py --dataset=pubmed --model=gee --dropout=0.9 --epochs=100 --hidden_dim=256 --layers=3 --lr=0.005 --weight_decay=0.009

  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Epoch: 010, Loss: 1.3342, Train: 0.3333, Val: 0.3880
Epoch: 020, Loss: 1.0823, Train: 0.5333, Val: 0.3320
Epoch: 030, Loss: 0.7905, Train: 0.3500, Val: 0.3900
Epoch: 040, Loss: 0.9855, Train: 0.5500, Val: 0.4200
Epoch: 050, Loss: 0.4475, Train: 0.6167, Val: 0.4320
Epoch: 060, Loss: 0.4776, Train: 0.7667, Val: 0.5300
Epoch: 070, Loss: 0.5439, Train: 0.9667, Val: 0.6000
Epoch: 080, Loss: 0.3718, Train: 1.0000, Val: 0.6760
Epoch: 090, Loss: 0.3356, Train: 1.0000, Val: 0.6900
Epoch: 100, Loss: 0.2727, Train: 1.0000, Val: 0.7120
Optimization Finished!
Training time: 35.3994s
Best validation accuracy: 0.7120
Test accuracy: 0.7120
  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Epoch:

In [21]:
!python train.py --dataset=wisconsin --model=gcn --dropout=0.9 --epochs=40 --hidden_dim=512 --layers=4 --lr=0.006 --weight_decay=0.009
!python train.py --dataset=wisconsin --model=gcn --dropout=0.9 --epochs=40 --hidden_dim=512 --layers=4 --lr=0.006 --weight_decay=0.009
!python train.py --dataset=wisconsin --model=gcn --dropout=0.9 --epochs=40 --hidden_dim=512 --layers=4 --lr=0.006 --weight_decay=0.009
!python train.py --dataset=wisconsin --model=gcn --dropout=0.9 --epochs=40 --hidden_dim=512 --layers=4 --lr=0.006 --weight_decay=0.009
!python train.py --dataset=wisconsin --model=gcn --dropout=0.9 --epochs=40 --hidden_dim=512 --layers=4 --lr=0.006 --weight_decay=0.009
!python train.py --dataset=wisconsin --model=gcn --dropout=0.9 --epochs=40 --hidden_dim=512 --layers=4 --lr=0.006 --weight_decay=0.009
!python train.py --dataset=wisconsin --model=gcn --dropout=0.9 --epochs=40 --hidden_dim=512 --layers=4 --lr=0.006 --weight_decay=0.009
!python train.py --dataset=wisconsin --model=gcn --dropout=0.9 --epochs=40 --hidden_dim=512 --layers=4 --lr=0.006 --weight_decay=0.009
!python train.py --dataset=wisconsin --model=gcn --dropout=0.9 --epochs=40 --hidden_dim=512 --layers=4 --lr=0.006 --weight_decay=0.009
!python train.py --dataset=wisconsin --model=gcn --dropout=0.9 --epochs=40 --hidden_dim=512 --layers=4 --lr=0.006 --weight_decay=0.009

Done loading data from cached files.
Epoch: 010, Loss: 1.2814, Train: 0.6167, Val: 0.5750
Epoch: 020, Loss: 1.0927, Train: 0.7167, Val: 0.6125
Epoch: 030, Loss: 1.0171, Train: 0.7917, Val: 0.5625
Epoch: 040, Loss: 0.8392, Train: 0.8500, Val: 0.5625
Optimization Finished!
Training time: 3.0962s
Best validation accuracy: 0.6125
Test accuracy: 0.6125
Done loading data from cached files.
Epoch: 010, Loss: 1.4582, Train: 0.6500, Val: 0.6125
Epoch: 020, Loss: 1.2711, Train: 0.7167, Val: 0.6000
Epoch: 030, Loss: 1.1845, Train: 0.7833, Val: 0.6000
Epoch: 040, Loss: 0.8097, Train: 0.8167, Val: 0.5750
Optimization Finished!
Training time: 3.1339s
Best validation accuracy: 0.6500
Test accuracy: 0.6500
Done loading data from cached files.
Epoch: 010, Loss: 1.4743, Train: 0.6250, Val: 0.5500
Epoch: 020, Loss: 1.1853, Train: 0.7083, Val: 0.6375
Epoch: 030, Loss: 0.9995, Train: 0.8000, Val: 0.6250
Epoch: 040, Loss: 0.8194, Train: 0.8417, Val: 0.5500
Optimization Finished!
Training time: 3.0390s
Best 

In [20]:
!python train.py --dataset=wisconsin --model=gat --dropout=0.75 --epochs=100 --hidden_dim=64 --layers=1 --lr=0.0015 --weight_decay=0.008 --heads=4
!python train.py --dataset=wisconsin --model=gat --dropout=0.75 --epochs=100 --hidden_dim=64 --layers=1 --lr=0.0015 --weight_decay=0.008 --heads=4
!python train.py --dataset=wisconsin --model=gat --dropout=0.75 --epochs=100 --hidden_dim=64 --layers=1 --lr=0.0015 --weight_decay=0.008 --heads=4
!python train.py --dataset=wisconsin --model=gat --dropout=0.75 --epochs=100 --hidden_dim=64 --layers=1 --lr=0.0015 --weight_decay=0.008 --heads=4
!python train.py --dataset=wisconsin --model=gat --dropout=0.75 --epochs=100 --hidden_dim=64 --layers=1 --lr=0.0015 --weight_decay=0.008 --heads=4
!python train.py --dataset=wisconsin --model=gat --dropout=0.75 --epochs=100 --hidden_dim=64 --layers=1 --lr=0.0015 --weight_decay=0.008 --heads=4
!python train.py --dataset=wisconsin --model=gat --dropout=0.75 --epochs=100 --hidden_dim=64 --layers=1 --lr=0.0015 --weight_decay=0.008 --heads=4
!python train.py --dataset=wisconsin --model=gat --dropout=0.75 --epochs=100 --hidden_dim=64 --layers=1 --lr=0.0015 --weight_decay=0.008 --heads=4
!python train.py --dataset=wisconsin --model=gat --dropout=0.75 --epochs=100 --hidden_dim=64 --layers=1 --lr=0.0015 --weight_decay=0.008 --heads=4
!python train.py --dataset=wisconsin --model=gat --dropout=0.75 --epochs=100 --hidden_dim=64 --layers=1 --lr=0.0015 --weight_decay=0.008 --heads=4

Done loading data from cached files.
Epoch: 010, Loss: 1.3723, Train: 0.4250, Val: 0.5000
Epoch: 020, Loss: 1.2979, Train: 0.6667, Val: 0.6250
Epoch: 030, Loss: 1.1926, Train: 0.6917, Val: 0.6250
Epoch: 040, Loss: 1.2782, Train: 0.7500, Val: 0.6000
Epoch: 050, Loss: 1.0590, Train: 0.7583, Val: 0.6000
Epoch: 060, Loss: 1.0987, Train: 0.7833, Val: 0.6000
Epoch: 070, Loss: 1.0533, Train: 0.7917, Val: 0.5875
Epoch: 080, Loss: 0.8772, Train: 0.8083, Val: 0.6000
Epoch: 090, Loss: 0.9204, Train: 0.8333, Val: 0.5875
Epoch: 100, Loss: 0.9375, Train: 0.8417, Val: 0.5750
Optimization Finished!
Training time: 1.5561s
Best validation accuracy: 0.6500
Test accuracy: 0.6500
Done loading data from cached files.
Epoch: 010, Loss: 1.4152, Train: 0.5000, Val: 0.5375
Epoch: 020, Loss: 1.2718, Train: 0.6583, Val: 0.6125
Epoch: 030, Loss: 1.2445, Train: 0.6750, Val: 0.5875
Epoch: 040, Loss: 1.1370, Train: 0.6750, Val: 0.6000
Epoch: 050, Loss: 1.0641, Train: 0.7667, Val: 0.6000
Epoch: 060, Loss: 1.0673, Trai

In [18]:
!python train.py --dataset=wisconsin --model=gee --dropout=0.7 --epochs=100 --hidden_dim=256 --layers=2 --lr=0.009 --weight_decay=0.006
!python train.py --dataset=wisconsin --model=gee --dropout=0.7 --epochs=100 --hidden_dim=256 --layers=2 --lr=0.009 --weight_decay=0.006
!python train.py --dataset=wisconsin --model=gee --dropout=0.7 --epochs=100 --hidden_dim=256 --layers=2 --lr=0.009 --weight_decay=0.006
!python train.py --dataset=wisconsin --model=gee --dropout=0.7 --epochs=100 --hidden_dim=256 --layers=2 --lr=0.009 --weight_decay=0.006
!python train.py --dataset=wisconsin --model=gee --dropout=0.7 --epochs=100 --hidden_dim=256 --layers=2 --lr=0.009 --weight_decay=0.006
!python train.py --dataset=wisconsin --model=gee --dropout=0.7 --epochs=100 --hidden_dim=256 --layers=2 --lr=0.009 --weight_decay=0.006
!python train.py --dataset=wisconsin --model=gee --dropout=0.7 --epochs=100 --hidden_dim=256 --layers=2 --lr=0.009 --weight_decay=0.006
!python train.py --dataset=wisconsin --model=gee --dropout=0.7 --epochs=100 --hidden_dim=256 --layers=2 --lr=0.009 --weight_decay=0.006
!python train.py --dataset=wisconsin --model=gee --dropout=0.7 --epochs=100 --hidden_dim=256 --layers=2 --lr=0.009 --weight_decay=0.006
!python train.py --dataset=wisconsin --model=gee --dropout=0.7 --epochs=100 --hidden_dim=256 --layers=2 --lr=0.009 --weight_decay=0.006

Done loading data from cached files.
Epoch: 010, Loss: 0.1138, Train: 0.9833, Val: 0.8000
Epoch: 020, Loss: 0.0694, Train: 0.9833, Val: 0.8375
Epoch: 030, Loss: 0.0110, Train: 1.0000, Val: 0.8375
Epoch: 040, Loss: 0.0102, Train: 1.0000, Val: 0.8500
Epoch: 050, Loss: 0.0142, Train: 1.0000, Val: 0.8375
Epoch: 060, Loss: 0.0107, Train: 1.0000, Val: 0.8500
Epoch: 070, Loss: 0.0033, Train: 1.0000, Val: 0.8500
Epoch: 080, Loss: 0.0014, Train: 1.0000, Val: 0.8375
Epoch: 090, Loss: 0.0057, Train: 1.0000, Val: 0.8375
Epoch: 100, Loss: 0.0021, Train: 1.0000, Val: 0.8375
Optimization Finished!
Training time: 1.8766s
Best validation accuracy: 0.8500
Test accuracy: 0.8500
Done loading data from cached files.
Epoch: 010, Loss: 0.1065, Train: 0.9750, Val: 0.8500
Epoch: 020, Loss: 0.0408, Train: 0.9833, Val: 0.8500
Epoch: 030, Loss: 0.0216, Train: 1.0000, Val: 0.8500
Epoch: 040, Loss: 0.0113, Train: 1.0000, Val: 0.8625
Epoch: 050, Loss: 0.0037, Train: 1.0000, Val: 0.8375
Epoch: 060, Loss: 0.0026, Trai

In [17]:
!python train.py --dataset=texas --model=gcn --dropout=0.2 --epochs=100 --hidden_dim=512 --layers=2 --lr=0.008 --weight_decay=0.008
!python train.py --dataset=texas --model=gcn --dropout=0.2 --epochs=100 --hidden_dim=512 --layers=2 --lr=0.008 --weight_decay=0.008
!python train.py --dataset=texas --model=gcn --dropout=0.2 --epochs=100 --hidden_dim=512 --layers=2 --lr=0.008 --weight_decay=0.008
!python train.py --dataset=texas --model=gcn --dropout=0.2 --epochs=100 --hidden_dim=512 --layers=2 --lr=0.008 --weight_decay=0.008
!python train.py --dataset=texas --model=gcn --dropout=0.2 --epochs=100 --hidden_dim=512 --layers=2 --lr=0.008 --weight_decay=0.008
!python train.py --dataset=texas --model=gcn --dropout=0.2 --epochs=100 --hidden_dim=512 --layers=2 --lr=0.008 --weight_decay=0.008
!python train.py --dataset=texas --model=gcn --dropout=0.2 --epochs=100 --hidden_dim=512 --layers=2 --lr=0.008 --weight_decay=0.008
!python train.py --dataset=texas --model=gcn --dropout=0.2 --epochs=100 --hidden_dim=512 --layers=2 --lr=0.008 --weight_decay=0.008
!python train.py --dataset=texas --model=gcn --dropout=0.2 --epochs=100 --hidden_dim=512 --layers=2 --lr=0.008 --weight_decay=0.008
!python train.py --dataset=texas --model=gcn --dropout=0.2 --epochs=100 --hidden_dim=512 --layers=2 --lr=0.008 --weight_decay=0.008

Done loading data from cached files.
Epoch: 010, Loss: 0.5303, Train: 0.8851, Val: 0.5932
Epoch: 020, Loss: 0.1522, Train: 0.9540, Val: 0.5593
Epoch: 030, Loss: 0.0442, Train: 1.0000, Val: 0.5932
Epoch: 040, Loss: 0.0331, Train: 1.0000, Val: 0.6610
Epoch: 050, Loss: 0.0112, Train: 1.0000, Val: 0.6780
Epoch: 060, Loss: 0.0084, Train: 1.0000, Val: 0.6271
Epoch: 070, Loss: 0.0036, Train: 1.0000, Val: 0.6441
Epoch: 080, Loss: 0.0060, Train: 1.0000, Val: 0.6780
Epoch: 090, Loss: 0.0021, Train: 1.0000, Val: 0.6610
Epoch: 100, Loss: 0.0050, Train: 1.0000, Val: 0.6949
Optimization Finished!
Training time: 4.8024s
Best validation accuracy: 0.6949
Test accuracy: 0.6949
Done loading data from cached files.
Epoch: 010, Loss: 0.5454, Train: 0.8851, Val: 0.5932
Epoch: 020, Loss: 0.0999, Train: 0.9655, Val: 0.5593
Epoch: 030, Loss: 0.0477, Train: 0.9770, Val: 0.6441
Epoch: 040, Loss: 0.0355, Train: 1.0000, Val: 0.6102
Epoch: 050, Loss: 0.0063, Train: 1.0000, Val: 0.6102
Epoch: 060, Loss: 0.0261, Trai

In [16]:
!python train.py --dataset=texas --model=gat --dropout=0.2 --epochs=100 --hidden_dim=512 --layers=5 --lr=0.002 --weight_decay=0.001 --heads=32
!python train.py --dataset=texas --model=gat --dropout=0.2 --epochs=100 --hidden_dim=512 --layers=5 --lr=0.002 --weight_decay=0.001 --heads=32
!python train.py --dataset=texas --model=gat --dropout=0.2 --epochs=100 --hidden_dim=512 --layers=5 --lr=0.002 --weight_decay=0.001 --heads=32
!python train.py --dataset=texas --model=gat --dropout=0.2 --epochs=100 --hidden_dim=512 --layers=5 --lr=0.002 --weight_decay=0.001 --heads=32
!python train.py --dataset=texas --model=gat --dropout=0.2 --epochs=100 --hidden_dim=512 --layers=5 --lr=0.002 --weight_decay=0.001 --heads=32
!python train.py --dataset=texas --model=gat --dropout=0.2 --epochs=100 --hidden_dim=512 --layers=5 --lr=0.002 --weight_decay=0.001 --heads=32
!python train.py --dataset=texas --model=gat --dropout=0.2 --epochs=100 --hidden_dim=512 --layers=5 --lr=0.002 --weight_decay=0.001 --heads=32
!python train.py --dataset=texas --model=gat --dropout=0.2 --epochs=100 --hidden_dim=512 --layers=5 --lr=0.002 --weight_decay=0.001 --heads=32
!python train.py --dataset=texas --model=gat --dropout=0.2 --epochs=100 --hidden_dim=512 --layers=5 --lr=0.002 --weight_decay=0.001 --heads=32
!python train.py --dataset=texas --model=gat --dropout=0.2 --epochs=100 --hidden_dim=512 --layers=5 --lr=0.002 --weight_decay=0.001 --heads=32

Done loading data from cached files.
Epoch: 010, Loss: 0.5549, Train: 0.8391, Val: 0.6441
Epoch: 020, Loss: 0.1483, Train: 0.9770, Val: 0.7288
Epoch: 030, Loss: 0.0584, Train: 0.9885, Val: 0.6441
Epoch: 040, Loss: 0.0222, Train: 1.0000, Val: 0.6780
Epoch: 050, Loss: 0.0078, Train: 1.0000, Val: 0.6441
Epoch: 060, Loss: 0.0083, Train: 1.0000, Val: 0.6441
Epoch: 070, Loss: 0.0039, Train: 1.0000, Val: 0.6610
Epoch: 080, Loss: 0.0010, Train: 1.0000, Val: 0.6780
Epoch: 090, Loss: 0.0005, Train: 1.0000, Val: 0.6949
Epoch: 100, Loss: 0.0003, Train: 1.0000, Val: 0.6949
Optimization Finished!
Training time: 19.7804s
Best validation accuracy: 0.7288
Test accuracy: 0.7288
Done loading data from cached files.
Epoch: 010, Loss: 0.4897, Train: 0.8161, Val: 0.6610
Epoch: 020, Loss: 0.1294, Train: 0.9655, Val: 0.5932
Epoch: 030, Loss: 0.0361, Train: 1.0000, Val: 0.6271
Epoch: 040, Loss: 0.0066, Train: 1.0000, Val: 0.6780
Epoch: 050, Loss: 0.0019, Train: 1.0000, Val: 0.6780
Epoch: 060, Loss: 0.0019, Tra

In [15]:
!python train.py --dataset=texas --model=gee --dropout=0.5 --epochs=100 --hidden_dim=512 --layers=4 --lr=0.008 --weight_decay=0.009
!python train.py --dataset=texas --model=gee --dropout=0.5 --epochs=100 --hidden_dim=512 --layers=4 --lr=0.008 --weight_decay=0.009
!python train.py --dataset=texas --model=gee --dropout=0.5 --epochs=100 --hidden_dim=512 --layers=4 --lr=0.008 --weight_decay=0.009
!python train.py --dataset=texas --model=gee --dropout=0.5 --epochs=100 --hidden_dim=512 --layers=4 --lr=0.008 --weight_decay=0.009
!python train.py --dataset=texas --model=gee --dropout=0.5 --epochs=100 --hidden_dim=512 --layers=4 --lr=0.008 --weight_decay=0.009
!python train.py --dataset=texas --model=gee --dropout=0.5 --epochs=100 --hidden_dim=512 --layers=4 --lr=0.008 --weight_decay=0.009
!python train.py --dataset=texas --model=gee --dropout=0.5 --epochs=100 --hidden_dim=512 --layers=4 --lr=0.008 --weight_decay=0.009
!python train.py --dataset=texas --model=gee --dropout=0.5 --epochs=100 --hidden_dim=512 --layers=4 --lr=0.008 --weight_decay=0.009
!python train.py --dataset=texas --model=gee --dropout=0.5 --epochs=100 --hidden_dim=512 --layers=4 --lr=0.008 --weight_decay=0.009
!python train.py --dataset=texas --model=gee --dropout=0.5 --epochs=100 --hidden_dim=512 --layers=4 --lr=0.008 --weight_decay=0.009

Done loading data from cached files.
Epoch: 010, Loss: 0.0143, Train: 1.0000, Val: 0.7119
Epoch: 020, Loss: 0.0046, Train: 1.0000, Val: 0.7288
Epoch: 030, Loss: 0.0004, Train: 1.0000, Val: 0.7288
Epoch: 040, Loss: 0.0063, Train: 1.0000, Val: 0.7627
Epoch: 050, Loss: 0.0001, Train: 1.0000, Val: 0.7797
Epoch: 060, Loss: 0.0003, Train: 1.0000, Val: 0.7627
Epoch: 070, Loss: 0.0026, Train: 1.0000, Val: 0.7627
Epoch: 080, Loss: 0.0001, Train: 1.0000, Val: 0.7966
Epoch: 090, Loss: 0.0002, Train: 1.0000, Val: 0.7797
Epoch: 100, Loss: 0.0000, Train: 1.0000, Val: 0.7797
Optimization Finished!
Training time: 4.4648s
Best validation accuracy: 0.7966
Test accuracy: 0.7966
Done loading data from cached files.
Epoch: 010, Loss: 0.0071, Train: 1.0000, Val: 0.6441
Epoch: 020, Loss: 0.0019, Train: 1.0000, Val: 0.7627
Epoch: 030, Loss: 0.0097, Train: 1.0000, Val: 0.8305
Epoch: 040, Loss: 0.0001, Train: 1.0000, Val: 0.8136
Epoch: 050, Loss: 0.0002, Train: 1.0000, Val: 0.7966
Epoch: 060, Loss: 0.0001, Trai